In [80]:
import pandas as pd
import numpy as np 

In [81]:
df = pd.read_csv('../data/urbanthreads_q4.csv')

In [82]:
df.head(3)

,Order ID,Order Date,Customer ID,Type,SKU,Category,Product Name,Unit Price,Qty,Total,...,Source,City,State,Warehouse,Del Date,Status,Return?,Reason,Ret Rec,Refund
0,UT-100001,2024-10-01,CUST_10001,New,TOP-CAS-M,Tops,Classic Cotton Tee,1299,2,2598,...,Website,Mumbai,Maharashtra,Mumbai,2024-10-05,Delivered,No,NaN,NaN,Not Applicable
1,UT-100002,2024-10-01,CUST_10002,Returning,DRS-FRM-S,Dresses,Floral Midi Dress,2499,1,2499,...,Mobile App,Delhi,Delhi,Delhi,2024-10-04,Delivered,Yes,Size Issue,2024-10-15,Processed
2,UT-100003,2024-01-10,CUST_10003,new,BOT-JNS-L,Bottoms,Slim Fit Jeans,1899,1,1899,...,Website,Bangalore,Karnataka,Bangalore,2024-10-06,Delivered,No,NaN,NaN,Not Applicable


In [83]:
df.shape

(346, 23)

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order ID       346 non-null    object
 1   Order Date     346 non-null    object
 2   Customer ID    346 non-null    object
 3   Type           346 non-null    object
 4   SKU            346 non-null    object
 5   Category       346 non-null    object
 6   Product Name   346 non-null    object
 7   Unit Price     346 non-null    int64 
 8   Qty            346 non-null    int64 
 9   Total          346 non-null    int64 
 10  Discount Code  140 non-null    object
 11  Disc %         346 non-null    int64 
 12  Payment        346 non-null    object
 13  Source         346 non-null    object
 14  City           346 non-null    object
 15  State          346 non-null    object
 16  Warehouse      346 non-null    object
 17  Del Date       328 non-null    object
 18  Status         346 non-null   

In [85]:
df.columns.tolist()

['Order ID',
 'Order Date',
 'Customer ID',
 'Type',
 'SKU',
 'Category',
 'Product Name',
 'Unit Price',
 'Qty',
 'Total',
 'Discount Code',
 'Disc %',
 'Payment',
 'Source',
 'City',
 'State',
 'Warehouse',
 'Del Date',
 'Status',
 'Return?',
 'Reason',
 'Ret Rec',
 'Refund']

In [86]:
cleaning_log = []

In [87]:
cleaning_log.append({
    "issue": " missing return_reason when return_initiated = No",
    "type": "expected",
    "action": "left as blank",
    
}
)

In [88]:
pd.DataFrame(cleaning_log)

,issue,type,action
0,missing return_reason when return_initiated = No,expected,left as blank


In [89]:
df_raw=df.copy()

In [90]:
df_raw.head(3)

,Order ID,Order Date,Customer ID,Type,SKU,Category,Product Name,Unit Price,Qty,Total,...,Source,City,State,Warehouse,Del Date,Status,Return?,Reason,Ret Rec,Refund
0,UT-100001,2024-10-01,CUST_10001,New,TOP-CAS-M,Tops,Classic Cotton Tee,1299,2,2598,...,Website,Mumbai,Maharashtra,Mumbai,2024-10-05,Delivered,No,NaN,NaN,Not Applicable
1,UT-100002,2024-10-01,CUST_10002,Returning,DRS-FRM-S,Dresses,Floral Midi Dress,2499,1,2499,...,Mobile App,Delhi,Delhi,Delhi,2024-10-04,Delivered,Yes,Size Issue,2024-10-15,Processed
2,UT-100003,2024-01-10,CUST_10003,new,BOT-JNS-L,Bottoms,Slim Fit Jeans,1899,1,1899,...,Website,Bangalore,Karnataka,Bangalore,2024-10-06,Delivered,No,NaN,NaN,Not Applicable


In [91]:
#remane the columns to make it more readable and easier to work with of data analysis and cleaning
column_rename_map = {
    "Order ID": "order_id",
    "Order Date": "order_date",
    "Customer ID": "customer_id",
    "Type": "customer_type",
    "SKU": "product_sku",
    "Category": "product_category",
    "Product Name": "product_name",
    "Unit Price": "unit_price",
    "Qty": "quantity",
    "Total": "order_total",
    "Discount Code": "discount_code",
    "Disc %": "discount_percent",
    "Payment": "payment_method",
    "Source": "order_source",
    "City": "shipping_city",
    "State": "shipping_state",
    "Warehouse": "warehouse_origin",
    "Del Date": "delivery_date",
    "Status": "delivery_status",
    "Return?": "return_initiated",
    "Reason": "return_reason",
    "Ret Rec": "return_received",
    "Refund": "refund_status"
}

df = df.rename(columns=column_rename_map)
df.columns.tolist()


['order_id',
 'order_date',
 'customer_id',
 'customer_type',
 'product_sku',
 'product_category',
 'product_name',
 'unit_price',
 'quantity',
 'order_total',
 'discount_code',
 'discount_percent',
 'payment_method',
 'order_source',
 'shipping_city',
 'shipping_state',
 'warehouse_origin',
 'delivery_date',
 'delivery_status',
 'return_initiated',
 'return_reason',
 'return_received',
 'refund_status']

In [92]:
#Missing value analysis 

missing_summary= (df.isnull().sum().reset_index().rename(columns={0:'missing_count'}))
missing_summary['missing_pct'] = (missing_summary["missing_count"]/len(df)*100).round(2)
missing_summary.sort_values("missing_count", ascending=False)



,index,missing_count,missing_pct
10,discount_code,206,59.54
21,return_received,166,47.98
20,return_reason,166,47.98
17,delivery_date,18,5.20
0,order_id,0,0.00
1,order_date,0,0.00
2,customer_id,0,0.00
6,product_name,0,0.00
5,product_category,0,0.00
4,product_sku,0,0.00


In [93]:
#lets categorize the columns before deceiding - Expected missing values, potential data quality issues, critical missing values
# to analyze further missing values 1- return reason is only applicable when return_initiated is yes
invalid_return_reason = df[
    (df["return_initiated"] == "No") &
    (df["return_reason"].notna())
]

invalid_return_reason.shape

(0, 23)

In [94]:
missing_return_received = df[
    (df["return_initiated"] == "Yes") &
    (df["return_received"].isna())
]

missing_return_received.shape


(0, 23)

In [95]:
cleaning_log.append({
    "column": "return_reason",
    "issue": "Missing values depend on return_initiated flag",
    "classification": "Expected & Invalid mixed",
    "decision": "Flag rows where return_initiated = Yes but return_reason is missing"
})


In [96]:
# deliver date has missing values when status is not delivered

missing_delivery_when_delivered = df[
    (df["delivery_status"]=="Delivered") &
    (df["delivery_date"].isna())
]

missing_delivery_when_delivered.shape

(0, 23)

In [97]:
#return received 
# Only applies when a return actually happened
# Should be blank when return_initiated = No

invalid_return_received = df[
    (df["return_initiated"]== "No" ) &
    (df["return_received"].notna())
]

print(invalid_return_received.shape)

missing_return_received = df[
    (df["return_initiated"]=="Yes") &
    (df["return_received"].isna())
]
print(missing_return_received.shape)

(0, 23)
(0, 23)


In [98]:
#refund status should not be missing when return is initiated
refund_issues = df[
    (df["return_initiated"] =="No") &
    (df["refund_status"].isna())
]

refund_issues.shape

(0, 23)

In [99]:
# #Discount related issues 

# discount_code
# Blank → no discount → Expected
# discount_percent
# Missing but discount_code exists → Issue
# Missing and no discount_code → may assume 0

discount_mismatch= df[
    (df["discount_code"].notna()) & 
    (df["discount_percent"].isna())
]

discount_mismatch.shape

(0, 23)

In [100]:
missing_decesions= pd.DataFrame(cleaning_log)
missing_decesions 


,issue,type,action,column,classification,decision
0,missing return_reason when return_initiated = No,expected,left as blank,NaN,NaN,NaN
1,Missing values depend on return_initiated flag,NaN,NaN,return_reason,Expected & Invalid mixed,Flag rows where return_initiated = Yes but ret...


In [103]:
missing_decesions.to_csv("../outputs/missing_value_log.csv", index=False) 
print(missing_decesions.shape)
print("Missing value log saved to ../outputs/missing_value_log.csv")


(2, 6)
Missing value log saved to ../outputs/missing_value_log.csv
